# ALL IMPORTS NECESSARY ARE HERE AT THE TOP

In [ ]:
import scipy.io as sio
import numpy as np
import pandas as pd
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.svm import SVC, SVR
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from google.colab import drive
from scipy.interpolate import interp1d
import random
from keras.models import Sequential
from keras.layers import Dense
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
!pip install tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score

drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 31.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.1
    Uninstalling numpy-1.25.1:
      Successfully uninstalled numpy-1.25.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.7 requires jax>=0.4.6, but you have jax 0.3.25 which is incompatible.
flax 0.7.0 requires jax>=0.4.2, but you have jax 0.3.25 which is incompatible.
orbax-checkpoint 0.2.7 requires jax>=0.4.9, but you have jax 0.3.25 which is incompatible.


Mounted at /content/drive


https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall?hl=pt-br

# READING THE DATA

In [ ]:
file_paths = [
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario1.mat',
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario2.mat',
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario3.mat',
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario4.mat',
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario5.mat',
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario6.mat',
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario7.mat',
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario8.mat',
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario9.mat'
]

file_data = []
df = []

for i, file_path in enumerate(file_paths):
    print(f"Reading File: {file_path}")
    data = sio.loadmat(file_path)
    df.append(pd.DataFrame(data['data'], columns=['ALE', 'NLE', 'ALD', 'NLD','S4_y1', 'S4_z1', 'S1_y1', 'S1_z1', 'S2_y1', 'S2_z1', 'S3_y1', 'S3_z1', 'Label']))

Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario1.mat
Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario2.mat
Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario3.mat
Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario4.mat
Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario5.mat
Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario6.mat
Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario7.mat
Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario8.mat
Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario9.mat


#RF REGRESSOR

#1%

In [ ]:
dataset1 = df[0]
dataset2 = df[1]
dataset3 = df[2]
dataset4 = df[3]
dataset5 = df[4]
dataset6 = df[5]
dataset7 = df[6]
dataset8 = df[7]
dataset9 = df[8]

dataset1 = dataset1[:10000]
dataset2 = dataset2[:10000]
dataset3 = dataset3[:10000]
dataset4 = dataset4[:10000]
dataset5 = dataset5[:10000]
dataset6 = dataset6[:10000]
dataset7 = dataset7[:10000]
dataset8 = dataset8[:10000]
dataset9 = dataset9[:10000]

print("Datasets read successfully")

Datasets read successfully


TRAIN STEP 1 REGRESSOR - RANDOM FOREST

In [ ]:
# Concatenate datasets 1, 2, 3, 4, 5, 7, 8, and 9 for training
#train_datasets = [dataset1_sample, dataset2_sample, dataset3_sample, dataset4_sample, dataset5_sample, dataset7_sample, dataset8_sample, dataset9_sample]

train_datasets = [dataset1, dataset2, dataset3, dataset4, dataset5, dataset6, dataset7, dataset8, dataset9]

train_data = pd.concat(train_datasets).reset_index(drop=True)

# Separate the training data into features (X) and irregularity data (y)
X = train_data.iloc[:, 4:-1]  # Only acceleration data
y = train_data[['ALE', 'NLE', 'ALD', 'NLD']]  # Irregularity data
z = train_data.iloc[:, -1]

# Create the scaler and scale the training features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = scaler.fit_transform(y)

X_train_scaled, X_test_scaled, y_train, y_test, z_train, z_test = train_test_split(X_scaled, y, z, test_size=0.1, random_state=42)

# Step 1: Train a Random Forest regression model for irregularity prediction
irregularity_model = RandomForestRegressor(n_estimators=50, random_state=42, verbose = 2)
irregularity_model.fit(X_train_scaled, y_train)

# Predict irregularity for the training data
irregularity_predictions = irregularity_model.predict(X_train_scaled)

# Calculate evaluation metrics for Step 1
mse = mean_squared_error(y_train, irregularity_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, irregularity_predictions)
r2 = r2_score(y_train, irregularity_predictions)

# Print the evaluation metrics for Step 1
print("Metrics for the RF Regressor - 1%")
print("RMSE:", rmse)
print("R2:", r2)
print("MAE:", mae)

# Step 2: Train an SVC model for damage prediction
X_train_combined = np.column_stack((X_train_scaled, irregularity_predictions))

building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  2.5min


building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    1.7s


Metrics for the RF Regressor - 1%
RMSE: 0.331939048644776
R2: 0.8897175100354501
MAE: 0.2303126413482636


In [ ]:
# Step 2: Train a LightGBM classifier for damage prediction
damage_model3 = SVC(kernel='rbf')
damage_model3.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model3.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.3s


Evaluation Metrics:
Precision: 0.2726
Recall: 0.4114
F1-score: 0.3279
Accuracy: 0.5814
True Positives (TP): 919
False Positives (FP): 2452
True Negatives (TN): 4314
False Negatives (FN): 1315


Classifier LightGBM

In [ ]:
# Step 2: Train a LightGBM classifier for damage prediction
damage_model3 = lgb.LGBMClassifier(n_estimators=50, random_state=42)
damage_model3.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model3.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

Evaluation Metrics:
Precision: 0.2955
Recall: 0.4123
F1-score: 0.3442
Accuracy: 0.6101
True Positives (TP): 921
False Positives (FP): 2196
True Negatives (TN): 4570
False Negatives (FN): 1313


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.2s


Classifier NEURAL NETWORK

In [ ]:
# Step 2: Train a neural network model for damage prediction
damage_model5 = Sequential()
damage_model5.add(Dense(64, activation='relu', input_dim=X_train_combined.shape[1]))
damage_model5.add(Dense(64, activation='relu'))
damage_model5.add(Dense(64, activation='relu'))
damage_model5.add(Dense(1, activation='sigmoid'))  # Output neuron for damage classification

damage_model5.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
damage_model5.fit(X_train_combined, z_train, epochs=10, batch_size=32)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model5.predict(X_test_combined)

# Convert damage probabilities to binary predictions using a threshold
threshold = 0.5
damage_predictions = (damage_predictions > threshold).astype(int)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

Epoch 1/10
2532/2532 [==============================] - 14s 3ms/step - loss: 0.0792 - accuracy: 0.9622
Epoch 2/10
2532/2532 [==============================] - 7s 3ms/step - loss: 0.0455 - accuracy: 0.9794
Epoch 3/10
2532/2532 [==============================] - 7s 3ms/step - loss: 0.0403 - accuracy: 0.9823
Epoch 4/10
2532/2532 [==============================] - 7s 3ms/step - loss: 0.0380 - accuracy: 0.9835
Epoch 5/10
2532/2532 [==============================] - 7s 3ms/step - loss: 0.0372 - accuracy: 0.9836
Epoch 6/10
2532/2532 [==============================] - 7s 3ms/step - loss: 0.0361 - accuracy: 0.9842
Epoch 7/10
2532/2532 [==============================] - 7s 3ms/step - loss: 0.0352 - accuracy: 0.9840
Epoch 8/10
2532/2532 [==============================] - 7s 3ms/step - loss: 0.0339 - accuracy: 0.9851
Epoch 9/10
2532/2532 [==============================] - 7s 3ms/step - loss: 0.0339 - accuracy: 0.9853
Epoch 10/10
  1/282 [..............................] - ETA: 23s

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.2s


282/282 [==============================] - 0s 1ms/step
Evaluation Metrics:
Precision: 0.2697
Recall: 0.4315
F1-score: 0.3319
Accuracy: 0.5688
True Positives (TP): 964
False Positives (FP): 2611
True Negatives (TN): 4155
False Negatives (FN): 1270


Classifier XGBoost

In [ ]:
# Step 2: Train an XGBoost classifier for damage prediction
damage_model2 = xgb.XGBClassifier(n_estimators=50, random_state=42)
damage_model2.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model2.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

Evaluation Metrics:
Precision: 0.2864
Recall: 0.4002
F1-score: 0.3339
Accuracy: 0.6037
True Positives (TP): 894
False Positives (FP): 2227
True Negatives (TN): 4539
False Negatives (FN): 1340


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.2s


Classifier RANDOM FOREST

In [ ]:
# Step 2: Train a Random Forest classifier for damage prediction
damage_model6 = RandomForestClassifier(n_estimators=50, random_state=42)
damage_model6.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model6.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

Evaluation Metrics:
Precision: 0.3089
Recall: 0.3563
F1-score: 0.3309
Accuracy: 0.6423
True Positives (TP): 796
False Positives (FP): 1781
True Negatives (TN): 4985
False Negatives (FN): 1438


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.2s


Classifier Thresholds

In [ ]:
# Predict irregularity for the testing data
irregularity_predictions = irregularity_model.predict(X_test_scaled)

# Compare the predicted irregularities with a threshold to predict damage
threshold_ale = 1.5e-3
threshold_nle = 2.3e-3
threshold_ald = 1.5e-3
threshold_nld = 2.3e-3

predicted_labels = []
for i in range(len(irregularity_predictions)):
    if (irregularity_predictions[i, 0] > threshold_ale) or (irregularity_predictions[i, 1] > threshold_nle) or (irregularity_predictions[i, 2] > threshold_ald) or (irregularity_predictions[i, 3] > threshold_nld):
        predicted_labels.append(1)
    else:
        predicted_labels.append(0)

# Extract the corresponding labels from the original dataset using the index of the test data
actual_labels = z_test.values

# Calculate evaluation metrics based on the predicted labels and the actual labels
accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, predicted_labels)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

Evaluation Metrics:
Precision: 0.2509
Recall: 0.9902
F1-score: 0.4003
Accuracy: 0.2637
True Positives (TP): 2212
False Positives (FP): 6605
True Negatives (TN): 161
False Negatives (FN): 22


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.2s


#5%

In [ ]:
dataset1 = df[0]
dataset2 = df[1]
dataset3 = df[2]
dataset4 = df[3]
dataset5 = df[4]
dataset6 = df[5]
dataset7 = df[6]
dataset8 = df[7]
dataset9 = df[8]

dataset1 = dataset1[:50000]
dataset2 = dataset2[:50000]
dataset3 = dataset3[:50000]
dataset4 = dataset4[:50000]
dataset5 = dataset5[:50000]
dataset6 = dataset6[:50000]
dataset7 = dataset7[:50000]
dataset8 = dataset8[:50000]
dataset9 = dataset9[:50000]

print("Datasets read successfully")

Datasets read successfully


TRAIN STEP 1 REGRESSOR - RANDOM FOREST

In [ ]:
# Concatenate datasets 1, 2, 3, 4, 5, 7, 8, and 9 for training
#train_datasets = [dataset1_sample, dataset2_sample, dataset3_sample, dataset4_sample, dataset5_sample, dataset7_sample, dataset8_sample, dataset9_sample]

train_datasets = [dataset1, dataset2, dataset3, dataset4, dataset5, dataset6, dataset7, dataset8, dataset9]

train_data = pd.concat(train_datasets).reset_index(drop=True)

# Separate the training data into features (X) and irregularity data (y)
X = train_data.iloc[:, 4:-1]  # Only acceleration data
y = train_data[['ALE', 'NLE', 'ALD', 'NLD']]  # Irregularity data
z = train_data.iloc[:, -1]

# Create the scaler and scale the training features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = scaler.fit_transform(y)

X_train_scaled, X_test_scaled, y_train, y_test, z_train, z_test = train_test_split(X_scaled, y, z, test_size=0.1, random_state=42)

# Step 1: Train a Random Forest regression model for irregularity prediction
irregularity_model = RandomForestRegressor(n_estimators=50, random_state=42, verbose = 2)
irregularity_model.fit(X_train_scaled, y_train)

# Predict irregularity for the training data
irregularity_predictions = irregularity_model.predict(X_train_scaled)

# Calculate evaluation metrics for Step 1
mse = mean_squared_error(y_train, irregularity_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, irregularity_predictions)
r2 = r2_score(y_train, irregularity_predictions)

# Print the evaluation metrics for Step 1
print("Metrics for the RF Regressor - 5%")
print("RMSE:", rmse)
print("R2:", r2)
print("MAE:", mae)

# Step 2: Train an SVC model for damage prediction
X_train_combined = np.column_stack((X_train_scaled, irregularity_predictions))

building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed: 10.8min


building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   18.2s


Metrics for the RF Regressor - 5%
RMSE: 0.3619960054096055
R2: 0.868874327052638
MAE: 0.25200585650517593


In [ ]:
# Step 2: Train a LightGBM classifier for damage prediction
damage_model3 = SVC(kernel='rbf')
damage_model3.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model3.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labelsp, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    2.9s


Evaluation Metrics:
Precision: 0.3551
Recall: 0.6650
F1-score: 0.4630
Accuracy: 0.4730
True Positives (TP): 10223
False Positives (FP): 18565
True Negatives (TN): 11063
False Negatives (FN): 5149


Classifier LightGBM

In [ ]:
# Step 2: Train a LightGBM classifier for damage prediction
damage_model3 = lgb.LGBMClassifier(n_estimators=50, random_state=42)
damage_model3.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model3.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    1.3s


Evaluation Metrics:
Precision: 0.3513
Recall: 0.6421
F1-score: 0.4542
Accuracy: 0.4728
True Positives (TP): 9870
False Positives (FP): 18222
True Negatives (TN): 11406
False Negatives (FN): 5502


Classifier NEURAL NETWORK

In [ ]:
# Step 2: Train a neural network model for damage prediction
damage_model5 = Sequential()
damage_model5.add(Dense(64, activation='relu', input_dim=X_train_combined.shape[1]))
damage_model5.add(Dense(64, activation='relu'))
damage_model5.add(Dense(64, activation='relu'))
damage_model5.add(Dense(1, activation='sigmoid'))  # Output neuron for damage classification

damage_model5.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
damage_model5.fit(X_train_combined, z_train, epochs=10, batch_size=32)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model5.predict(X_test_combined)

# Convert damage probabilities to binary predictions using a threshold
threshold = 0.5
damage_predictions = (damage_predictions > threshold).astype(int)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

Epoch 1/10
12657/12657 [==============================] - 38s 3ms/step - loss: 0.1549 - accuracy: 0.9337
Epoch 2/10
12657/12657 [==============================] - 36s 3ms/step - loss: 0.1142 - accuracy: 0.9529
Epoch 3/10
12657/12657 [==============================] - 36s 3ms/step - loss: 0.1055 - accuracy: 0.9569
Epoch 4/10
12657/12657 [==============================] - 36s 3ms/step - loss: 0.1005 - accuracy: 0.9588
Epoch 5/10
12657/12657 [==============================] - 36s 3ms/step - loss: 0.0971 - accuracy: 0.9602
Epoch 6/10
12657/12657 [==============================] - 36s 3ms/step - loss: 0.0941 - accuracy: 0.9615
Epoch 7/10
12657/12657 [==============================] - 36s 3ms/step - loss: 0.0921 - accuracy: 0.9621
Epoch 8/10
12657/12657 [==============================] - 36s 3ms/step - loss: 0.0899 - accuracy: 0.9631
Epoch 9/10
12657/12657 [==============================] - 36s 3ms/step - loss: 0.0885 - accuracy: 0.9640
Epoch 10/10
12657/12657 [==============================

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    1.3s


1407/1407 [==============================] - 2s 1ms/step
Evaluation Metrics:
Precision: 0.3456
Recall: 0.6652
F1-score: 0.4549
Accuracy: 0.4554
True Positives (TP): 10225
False Positives (FP): 19360
True Negatives (TN): 10268
False Negatives (FN): 5147


Classifier XGBoost

In [ ]:
# Step 2: Train an XGBoost classifier for damage prediction
damage_model2 = xgb.XGBClassifier(n_estimators=50, random_state=42)
damage_model2.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model2.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    1.3s


Evaluation Metrics:
Precision: 0.3500
Recall: 0.6672
F1-score: 0.4592
Accuracy: 0.4631
True Positives (TP): 10256
False Positives (FP): 19045
True Negatives (TN): 10583
False Negatives (FN): 5116


Classifier RANDOM FOREST

In [ ]:
# Step 2: Train a Random Forest classifier for damage prediction
damage_model6 = RandomForestClassifier(n_estimators=50, random_state=42)
damage_model6.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model6.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    1.3s


Evaluation Metrics:
Precision: 0.3502
Recall: 0.6458
F1-score: 0.4541
Accuracy: 0.4696
True Positives (TP): 9928
False Positives (FP): 18422
True Negatives (TN): 11206
False Negatives (FN): 5444


Classifier Thresholds

In [ ]:
# Predict irregularity for the testing data
irregularity_predictions = irregularity_model.predict(X_test_scaled)

# Compare the predicted irregularities with a threshold to predict damage
threshold_ale = 1.5e-3
threshold_nle = 2.3e-3
threshold_ald = 1.5e-3
threshold_nld = 2.3e-3

predicted_labels = []
for i in range(len(irregularity_predictions)):
    if (irregularity_predictions[i, 0] > threshold_ale) or (irregularity_predictions[i, 1] > threshold_nle) or (irregularity_predictions[i, 2] > threshold_ald) or (irregularity_predictions[i, 3] > threshold_nld):
        predicted_labels.append(1)
    else:
        predicted_labels.append(0)

# Extract the corresponding labels from the original dataset using the index of the test data
actual_labels = z_test.values

# Calculate evaluation metrics based on the predicted labels and the actual labels
accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, predicted_labels)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    2.0s


Evaluation Metrics:
Precision: 0.3739
Recall: 0.8507
F1-score: 0.5195
Accuracy: 0.4625
True Positives (TP): 13077
False Positives (FP): 21893
True Negatives (TN): 7735
False Negatives (FN): 2295


#25%

In [ ]:
dataset1 = df[0]
dataset2 = df[1]
dataset3 = df[2]
dataset4 = df[3]
dataset5 = df[4]
dataset6 = df[5]
dataset7 = df[6]
dataset8 = df[7]
dataset9 = df[8]

dataset1 = dataset1[:250000]
dataset2 = dataset2[:250000]
dataset3 = dataset3[:250000]
dataset4 = dataset4[:250000]
dataset5 = dataset5[:250000]
dataset6 = dataset6[:250000]
dataset7 = dataset7[:250000]
dataset8 = dataset8[:250000]
dataset9 = dataset9[:250000]

print("Datasets read successfully")

Datasets read successfully


TRAIN STEP 1 REGRESSOR - RANDOM FOREST

In [ ]:
# Concatenate datasets 1, 2, 3, 4, 5, 7, 8, and 9 for training
#train_datasets = [dataset1_sample, dataset2_sample, dataset3_sample, dataset4_sample, dataset5_sample, dataset7_sample, dataset8_sample, dataset9_sample]

train_datasets = [dataset1, dataset2, dataset3, dataset4, dataset5, dataset6, dataset7, dataset8, dataset9]

train_data = pd.concat(train_datasets).reset_index(drop=True)

# Separate the training data into features (X) and irregularity data (y)
X = train_data.iloc[:, 4:-1]  # Only acceleration data
y = train_data[['ALE', 'NLE', 'ALD', 'NLD']]  # Irregularity data
z = train_data.iloc[:, -1]

# Create the scaler and scale the training features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = scaler.fit_transform(y)

X_train_scaled, X_test_scaled, y_train, y_test, z_train, z_test = train_test_split(X_scaled, y, z, test_size=0.1, random_state=42)

# Step 1: Train a Random Forest regression model for irregularity prediction
irregularity_model = RandomForestRegressor(n_estimators=50, random_state=42, verbose = 2)
irregularity_model.fit(X_train_scaled, y_train)

# Predict irregularity for the training data
irregularity_predictions = irregularity_model.predict(X_train_scaled)

# Calculate evaluation metrics for Step 1
mse = mean_squared_error(y_train, irregularity_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, irregularity_predictions)
r2 = r2_score(y_train, irregularity_predictions)

# Print the evaluation metrics for Step 1
print("Metrics for the RF Regressor - 25%")
print("RMSE:", rmse)
print("R2:", r2)
print("MAE:", mae)

# Step 2: Train an SVC model for damage prediction
X_train_combined = np.column_stack((X_train_scaled, irregularity_predictions))

building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed: 40.9min


building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  1.4min


Metrics for the RF Regressor - 25%
RMSE: 0.36502539569886366
R2: 0.8667350321786691
MAE: 0.26980613835494294


Classifier LightGBM

In [ ]:
# Step 2: Train a LightGBM classifier for damage prediction
damage_model3 = lgb.LGBMClassifier(n_estimators=50, random_state=42)
damage_model3.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model3.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    9.5s


Evaluation Metrics:
Precision: 0.4253
Recall: 0.5913
F1-score: 0.4948
Accuracy: 0.5137
True Positives (TP): 53579
False Positives (FP): 72393
True Negatives (TN): 61993
False Negatives (FN): 37035


Classifier NEURAL NETWORK

In [ ]:
# Step 2: Train a neural network model for damage prediction
damage_model5 = Sequential()
damage_model5.add(Dense(64, activation='relu', input_dim=X_train_combined.shape[1]))
damage_model5.add(Dense(64, activation='relu'))
damage_model5.add(Dense(64, activation='relu'))
damage_model5.add(Dense(1, activation='sigmoid'))  # Output neuron for damage classification

damage_model5.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
damage_model5.fit(X_train_combined, z_train, epochs=10, batch_size=32)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model5.predict(X_test_combined)

# Convert damage probabilities to binary predictions using a threshold
threshold = 0.5
damage_predictions = (damage_predictions > threshold).astype(int)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

Epoch 1/10
63282/63282 [==============================] - 185s 3ms/step - loss: 0.1448 - accuracy: 0.9376
Epoch 2/10
63282/63282 [==============================] - 183s 3ms/step - loss: 0.1211 - accuracy: 0.9484
Epoch 3/10
63282/63282 [==============================] - 184s 3ms/step - loss: 0.1155 - accuracy: 0.9510
Epoch 4/10
63282/63282 [==============================] - 189s 3ms/step - loss: 0.1125 - accuracy: 0.9525
Epoch 5/10
63282/63282 [==============================] - 192s 3ms/step - loss: 0.1104 - accuracy: 0.9534
Epoch 6/10
63282/63282 [==============================] - 193s 3ms/step - loss: 0.1088 - accuracy: 0.9542
Epoch 7/10
63282/63282 [==============================] - 190s 3ms/step - loss: 0.1075 - accuracy: 0.9548
Epoch 8/10
63282/63282 [==============================] - 192s 3ms/step - loss: 0.1065 - accuracy: 0.9552
Epoch 9/10
63282/63282 [==============================] - 192s 3ms/step - loss: 0.1058 - accuracy: 0.9555
Epoch 10/10
63282/63282 [=====================

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   10.7s


7032/7032 [==============================] - 9s 1ms/step
Evaluation Metrics:
Precision: 0.4162
Recall: 0.6261
F1-score: 0.5000
Accuracy: 0.4958
True Positives (TP): 56730
False Positives (FP): 79564
True Negatives (TN): 54822
False Negatives (FN): 33884


Classifier XGBoost

In [ ]:
# Step 2: Train an XGBoost classifier for damage prediction
damage_model2 = xgb.XGBClassifier(n_estimators=50, random_state=42)
damage_model2.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model2.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   10.6s


Evaluation Metrics:
Precision: 0.4235
Recall: 0.5898
F1-score: 0.4930
Accuracy: 0.5114
True Positives (TP): 53447
False Positives (FP): 72762
True Negatives (TN): 61624
False Negatives (FN): 37167


Classifier RANDOM FOREST

In [ ]:
# Step 2: Train a Random Forest classifier for damage prediction
damage_model6 = RandomForestClassifier(n_estimators=50, random_state=42)
damage_model6.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model6.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   10.4s


Evaluation Metrics:
Precision: 0.4219
Recall: 0.5953
F1-score: 0.4938
Accuracy: 0.5085
True Positives (TP): 53942
False Positives (FP): 73917
True Negatives (TN): 60469
False Negatives (FN): 36672


Classifier Thresholds

In [ ]:
# Predict irregularity for the testing data
irregularity_predictions = irregularity_model.predict(X_test_scaled)

# Compare the predicted irregularities with a threshold to predict damage
threshold_ale = 1.5e-3
threshold_nle = 2.3e-3
threshold_ald = 1.5e-3
threshold_nld = 2.3e-3

predicted_labels = []
for i in range(len(irregularity_predictions)):
    if (irregularity_predictions[i, 0] > threshold_ale) or (irregularity_predictions[i, 1] > threshold_nle) or (irregularity_predictions[i, 2] > threshold_ald) or (irregularity_predictions[i, 3] > threshold_nld):
        predicted_labels.append(1)
    else:
        predicted_labels.append(0)

# Extract the corresponding labels from the original dataset using the index of the test data
actual_labels = z_test.values

# Calculate evaluation metrics based on the predicted labels and the actual labels
accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, predicted_labels)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   10.5s


Evaluation Metrics:
Precision: 0.4426
Recall: 0.8338
F1-score: 0.5782
Accuracy: 0.5101
True Positives (TP): 75551
False Positives (FP): 95156
True Negatives (TN): 39230
False Negatives (FN): 15063


#100%

In [ ]:
dataset1 = df[0]
dataset2 = df[1]
dataset3 = df[2]
dataset4 = df[3]
dataset5 = df[4]
dataset6 = df[5]
dataset7 = df[6]
dataset8 = df[7]
dataset9 = df[8]

dataset1 = dataset1[:1000000]
dataset2 = dataset2[:1000000]
dataset3 = dataset3[:1000000]
dataset4 = dataset4[:1000000]
dataset5 = dataset5[:1000000]
dataset6 = dataset6[:1000000]
dataset7 = dataset7[:1000000]
dataset8 = dataset8[:1000000]
dataset9 = dataset9[:1000000]

print("Datasets read successfully")

Datasets read successfully


TRAIN STEP 1 REGRESSOR - RANDOM FOREST

In [ ]:
# Concatenate datasets 1, 2, 3, 4, 5, 7, 8, and 9 for training
#train_datasets = [dataset1_sample, dataset2_sample, dataset3_sample, dataset4_sample, dataset5_sample, dataset7_sample, dataset8_sample, dataset9_sample]

train_datasets = [dataset1, dataset2, dataset3, dataset4, dataset5, dataset6, dataset7, dataset8, dataset9]

train_data = pd.concat(train_datasets).reset_index(drop=True)

# Separate the training data into features (X) and irregularity data (y)
X = train_data.iloc[:, 4:-1]  # Only acceleration data
y = train_data[['ALE', 'NLE', 'ALD', 'NLD']]  # Irregularity data
z = train_data.iloc[:, -1]

# Create the scaler and scale the training features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = scaler.fit_transform(y)

X_train_scaled, X_test_scaled, y_train, y_test, z_train, z_test = train_test_split(X_scaled, y, z, test_size=0.1, random_state=42)

# Step 1: Train a Random Forest regression model for irregularity prediction
irregularity_model = RandomForestRegressor(n_estimators=30, random_state=42, verbose = 2)
irregularity_model.fit(X_train_scaled, y_train)

# Predict irregularity for the training data
irregularity_predictions = irregularity_model.predict(X_train_scaled)

# Calculate evaluation metrics for Step 1
mse = mean_squared_error(y_train, irregularity_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, irregularity_predictions)
r2 = r2_score(y_train, irregularity_predictions)

# Print the evaluation metrics for Step 1
print("Metrics for the RF Regressor - 100%")
print("RMSE:", rmse)
print("R2:", r2)
print("MAE:", mae)

# Step 2: Train an SVC model for damage prediction
X_train_combined = np.column_stack((X_train_scaled, irregularity_predictions))

building tree 1 of 30
building tree 2 of 30
building tree 3 of 30
building tree 4 of 30
building tree 5 of 30
building tree 6 of 30
building tree 7 of 30
building tree 8 of 30
building tree 9 of 30
building tree 10 of 30
building tree 11 of 30
building tree 12 of 30
building tree 13 of 30
building tree 14 of 30
building tree 15 of 30
building tree 16 of 30
building tree 17 of 30
building tree 18 of 30
building tree 19 of 30
building tree 20 of 30
building tree 21 of 30
building tree 22 of 30
building tree 23 of 30
building tree 24 of 30
building tree 25 of 30
building tree 26 of 30
building tree 27 of 30
building tree 28 of 30
building tree 29 of 30
building tree 30 of 30
Metrics for the RF Regressor - 100%
RMSE: 0.3731724398604293
R2: 0.8607334116267602
MAE: 0.2856319892912499


Classifier LightGBM

In [ ]:
# Step 2: Train a LightGBM classifier for damage prediction
damage_model3 = lgb.LGBMClassifier(n_estimators=50, random_state=42)
damage_model3.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model3.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

Evaluation Metrics:
Precision: 0.4443
Recall: 0.4402
F1-score: 0.4422
Accuracy: 0.5466
True Positives (TP): 161761
False Positives (FP): 202354
True Negatives (TN): 330143
False Negatives (FN): 205742


Classifier NEURAL NETWORK

In [ ]:
# Step 2: Train a neural network model for damage prediction
damage_model5 = Sequential()
damage_model5.add(Dense(64, activation='relu', input_dim=X_train_combined.shape[1]))
damage_model5.add(Dense(64, activation='relu'))
damage_model5.add(Dense(64, activation='relu'))
damage_model5.add(Dense(1, activation='sigmoid'))  # Output neuron for damage classification

damage_model5.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
damage_model5.fit(X_train_combined, z_train, epochs=10, batch_size=32)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model5.predict(X_test_combined)

# Convert damage probabilities to binary predictions using a threshold
threshold = 0.5
damage_predictions = (damage_predictions > threshold).astype(int)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

Epoch 1/10
253125/253125 [==============================] - 378s 1ms/step - loss: 0.1552 - accuracy: 0.9319
Epoch 2/10
253125/253125 [==============================] - 374s 1ms/step - loss: 0.1423 - accuracy: 0.9383
Epoch 3/10
253125/253125 [==============================] - 374s 1ms/step - loss: 0.1396 - accuracy: 0.9396
Epoch 4/10
253125/253125 [==============================] - 370s 1ms/step - loss: 0.1385 - accuracy: 0.9403
Epoch 5/10
253125/253125 [==============================] - 370s 1ms/step - loss: 0.1378 - accuracy: 0.9407
Epoch 6/10
253125/253125 [==============================] - 369s 1ms/step - loss: 0.1374 - accuracy: 0.9410
Epoch 7/10
253125/253125 [==============================] - 368s 1ms/step - loss: 0.1372 - accuracy: 0.9411
Epoch 8/10
253125/253125 [==============================] - 364s 1ms/step - loss: 0.1373 - accuracy: 0.9413
Epoch 9/10
253125/253125 [==============================] - 365s 1ms/step - loss: 0.1385 - accuracy: 0.9411
Epoch 10/10
28125/28125 [===

Classifier XGBoost

In [ ]:
# Step 2: Train an XGBoost classifier for damage prediction
damage_model2 = xgb.XGBClassifier(n_estimators=50, random_state=42)
damage_model2.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model2.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

Evaluation Metrics:
Precision: 0.4394
Recall: 0.4452
F1-score: 0.4423
Accuracy: 0.5415
True Positives (TP): 163622
False Positives (FP): 208740
True Negatives (TN): 323757
False Negatives (FN): 203881


Classifier RANDOM FOREST

In [ ]:
# Step 2: Train a Random Forest classifier for damage prediction
damage_model6 = RandomForestClassifier(n_estimators=50, random_state=42)
damage_model6.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model6.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

Evaluation Metrics:
Precision: 0.4404
Recall: 0.4491
F1-score: 0.4447
Accuracy: 0.5420
True Positives (TP): 165033
False Positives (FP): 209692
True Negatives (TN): 322805
False Negatives (FN): 202470


Classifier Thresholds

In [ ]:
# Predict irregularity for the testing data
irregularity_predictions = irregularity_model.predict(X_test_scaled)

# Compare the predicted irregularities with a threshold to predict damage
threshold_ale = 1.5e-3
threshold_nle = 2.3e-3
threshold_ald = 1.5e-3
threshold_nld = 2.3e-3

predicted_labels = []
for i in range(len(irregularity_predictions)):
    if (irregularity_predictions[i, 0] > threshold_ale) or (irregularity_predictions[i, 1] > threshold_nle) or (irregularity_predictions[i, 2] > threshold_ald) or (irregularity_predictions[i, 3] > threshold_nld):
        predicted_labels.append(1)
    else:
        predicted_labels.append(0)

# Extract the corresponding labels from the original dataset using the index of the test data
actual_labels = z_test.values

# Calculate evaluation metrics based on the predicted labels and the actual labels
accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, predicted_labels)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

Evaluation Metrics:
Precision: 0.4581
Recall: 0.8168
F1-score: 0.5870
Accuracy: 0.5307
True Positives (TP): 300160
False Positives (FP): 355029
True Negatives (TN): 177468
False Negatives (FN): 67343
